# Data Analytics Project Work
### File for testing different stuff for the project :D

In [151]:
# imports
import pandas as pd
import numpy as np

# Testing on reading one of the datasets and creating a Pandas DataFrame - object
test_df = pd.read_csv("HybridDataset_for_RSSbasedLocalization-main/Scenario 0/RPI/RSS_BLT_Dataset/Distanza10.csv", sep=';')
test_df.head()

,Timestamp,Rx Power [dBm]
0,Fri Mar 24 14:13:33 2023,-79
1,Fri Mar 24 14:13:34 2023,-70
2,Fri Mar 24 14:13:36 2023,-73
3,Fri Mar 24 14:13:38 2023,-79
4,Fri Mar 24 14:13:39 2023,-76


In [152]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Timestamp       517 non-null    object
 1   Rx Power [dBm]  517 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.2+ KB


Everything seemed to be working great as we got the first five entries of the dataset with a .head()– method and some information about the dataset with the .info()– method.

In [153]:
def read_data(anchors, directory_path):
    """
    Reads the csv-files for one specified measurement case and adds them into a dict of dataframes.
    
    Parameters:
    anchors (list): List of anchors, for example ["Anchor 1", "Anchor 2", "Anchor 3"...]
    directory path (string): The relative directory path of the folder which contains the wanted anchor-csv.files.

    returns: 
    dict: a dict which keys are the anchor names and values are the respective dataframes
    """
    scenario_dataframes = {}

    for anchor in anchors:
        current_df = pd.read_csv(directory_path + anchor + ".csv", sep=';')
        scenario_dataframes[anchor.replace(".csv","")] = current_df
    
    return scenario_dataframes


In [154]:
def calculate_mean_values(scenario_dataframe, bursts):
    """
    Calculates the mean value over a given amount of bursts.

    Parameters:
    scenario_dataframe (DataFrame): A DataFrame-object containing the dataset of one anchor.
    bursts (int): The amount of bursts.

    Returns:
    int: Returns the mean value over the given amount of bursts.
    """

    mean_values = []

    power_values = scenario_dataframe["Rx Power [dBm]"].to_list()
    list_size = len(power_values)
    burst_size = list_size // bursts

    start = 0
    for i in range(bursts):
        end = start + burst_size if i < bursts - 1 else list_size - 1
        mean_values.append(power_values[end])
        start = end
        
    mean_value = round(np.mean(mean_values), 4)
    
    return mean_value

In [155]:
def calculate_estimates(p_initial, n_value, mean_values):
    """
    Calculates the distance estimates for each anchor.

    Parameteres:
    p_initial (int): The initial RSS value for the distance estimation equation.
    n_value (int): The n_value for the distance estimation formula.
    mean_values (dict): A dict which keys are the anchor names and values are their respective mean values over the given amount of bursts

    Returns:
    dict: Returns a dict which keys are the anchor names and values are their respective distance estimations.
    """
    estimated_distances = {}

    for anchor in mean_values:
        estimated_d = 10**((p_initial-mean_values[anchor])/(10*n_value))
        estimated_distances[anchor] = estimated_d

    return estimated_distances

Let's estimate the distances of the anchors in Scenario A RPI BLT_dataset

In [156]:
anchors = ["Anchor 1", "Anchor 2", "Anchor 3", "Anchor 4", "Anchor 5", "Anchor 6"]
directory_path = "HybridDataset_for_RSSbasedLocalization-main/Scenario A/RPI/RSS_BLT_Dataset/"

dataframes = read_data(anchors, directory_path)
bursts = 10

mean_values = {}
for anchor in dataframes:
    mean_values[anchor] = calculate_mean_values(dataframes[anchor], bursts)
    print(anchor, "mean value over", bursts, "bursts:", mean_values[anchor])

p_initial = -40
n_value = 2.8
estimated_distances = calculate_estimates(p_initial, n_value, mean_values)

print("")
for anchor in anchors:
    print(anchor, "Estimated distance:", estimated_distances[anchor])

Anchor 1 mean value over 10 bursts: -71.5
Anchor 2 mean value over 10 bursts: -75.3
Anchor 3 mean value over 10 bursts: -71.0
Anchor 4 mean value over 10 bursts: -71.5
Anchor 5 mean value over 10 bursts: -72.6
Anchor 6 mean value over 10 bursts: -60.1

Anchor 1 Estimated distance: 13.33521432163324
Anchor 2 Estimated distance: 18.22696189772763
Anchor 3 Estimated distance: 12.798022139979539
Anchor 4 Estimated distance: 13.33521432163324
Anchor 5 Estimated distance: 14.597743028861673
Anchor 6 Estimated distance: 5.222243798756885
